In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 200

In [ ]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))

In [ ]:
data = pd.read_parquet('../data/all_data.parquet')

In [ ]:
data.info(memory_usage='deep')

In [ ]:
titles = pd.read_csv('../data/titles_list_data.csv', low_memory=False, index_col=[0])

In [ ]:
titles.title.duplicated().value_counts()
# there are identical titles so we have to connect them with a unique id for convenience

In [ ]:
data.issue_link[0]

In [ ]:
# extract titles' and issues' unique ids
data['title_id'] = data.title_link.str.extract(pat=r'/series.(.+\d)/\d.+')
data['issue_id'] = data.issue_link.str.extract(pat=r'/series/.+\d/(.+\d)/.+')

In [ ]:
# keep relevant columns
df = data[['pub_name', 'title', 'title_id', 'issue', 'variant_of', 'cover_date', 
           'years', 'cover_price', 'current_value', 'searched', 
          'owned', 'volume', 'issues_total',
           'pub_titles_total', 'pub_issues_total',
           'contributors_names', 'contributors_roles', 'characters']]

In [ ]:
# turn prices into numerical
df.loc[:, 'cover_price'] = df.loc[:, 'cover_price'].str.replace('[$,]', '', regex=True)
df.loc[:, 'cover_price'] = df.loc[:, 'cover_price'].str.replace('Free', '0', regex=False)
df.loc[:, 'cover_price'] = df.loc[:, 'cover_price'].apply(eval)

In [ ]:
df.loc[:, 'current_value'] = df.loc[:, 'current_value'].str.replace('[$,]', '', regex=True)
df.loc[:, 'current_value'] = df.loc[:, 'current_value'].str.replace('Free', '0', regex=False)
df.loc[:, 'current_value'] = df.loc[:, 'current_value'].apply(eval)

In [ ]:
# Many issues with cover price 0 have high current value so we keep them
df[df.cover_price == 0].sort_values('current_value', ascending=False).head(5)

### `issue` column

There are several types of issues as indicated by their numbering and many strange numberings:

* Simple ascending numbering # 1-...
* 'Ashcan' Issues which are traditionally used to promote a new series (see [wiki](https://en.wikipedia.org/wiki/Ashcan_comic)). Ashcan comics can be quite rare and valuable, especially from the [Golden Age](https://en.wikipedia.org/wiki/Golden_Age_of_Comic_Books) (30s-50s)
* Issues with `nn` meaning 'non-numbered'. Typically 0th issues or one-shots
* Issues with numbering of the format: # (number)(variant-cover/ special ed./ convention ed. etc.): Normally these issues should be indicated as "variant of" (issue number) in the `variant_of` column
* Of "Vol ** # **" format (where ** indicated a number). i.e. two enumerations are indicated, one for volume, one for issue number
* Instead of issue number a date is given e.g. "Spring 2007" or simply "1989"
* Numbering which includes letters e.g. "C-2" or "x"
* Just the name of the protagonist or in general something explanatory e.g. "Batwoman"
* The print run is indicated e.g. '# 2 - 2nd print'


In [ ]:
# Let's get rid of 

In [ ]:
df.loc[:, 'proper_date'] = df.loc[:, 'cover_date']

In [ ]:
df.loc[:, 'proper_date'][df['proper_date'].isnull()] = df['years'][df['proper_date'].isnull()].str[:4]

In [ ]:
df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace('Winter ', '', regex=False)
df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace('Early ', '', regex=False)
df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace('Spring ', '', regex=False)
df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace('Holiday ', '', regex=False)
df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace('Summer ', '', regex=False)
df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace('Late ', '', regex=False)
df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace('Fall ', '', regex=False)
df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace('Mid ', '', regex=False)

In [ ]:
df.loc[df['proper_date'] == 'Spring','proper_date'] = np.nan
df.loc[df['proper_date'] == 'Unda','proper_date'] = np.nan
df.loc[df['proper_date'] == 'Unkn','proper_date'] = np.nan
df.loc[df['proper_date'] == 'Winter','proper_date'] = np.nan
df.loc[df['proper_date'] == 'Early','proper_date'] = np.nan
df.loc[df['proper_date'] == 'Late','proper_date'] = np.nan
df.loc[df['proper_date'] == 'No D','proper_date'] = np.nan
df.loc[df['proper_date'] == '199','proper_date'] = '1990'

In [ ]:
for i in range(30,100):
    oldvalue = "'"+str(i)
    newvalue = '19'+str(i)
    df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace(oldvalue, newvalue, regex=False)

In [ ]:
for i in range(0,10):
    oldvalue = "'0"+str(i)
    newvalue = '200'+str(i)
    df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace(oldvalue, newvalue, regex=False)

In [ ]:
for i in range(10,23):
    oldvalue = "'"+str(i)
    newvalue = '20'+str(i)
    df.loc[:, 'proper_date'] = df.loc[:, 'proper_date'].str.replace(oldvalue, newvalue, regex=False)

In [ ]:
df.loc[:, 'proper_date'] = pd.to_datetime(df['proper_date'])